# Тестирование

## Виды тестирования

- **Модульные тесты (Unit tests)** - проверка на корректность всех компонентов проекта по отдельности. Пишем отдельный тест для каждой нетривиальной функции/метода. 
- **Интеграционные тесты (Integration tests)** - проверка на работоспособность проекта в целом. Все компоненты собираются вместе (код, база данных/файловая система/любая другая третья сторона) и проверяются в разных средах (разные ос/версии питона и т.д.). Более сложная настройка - подготовить среду тестирования, установить все зависимости. 
- **Функциональные тесты (Functional tests)** - проверка работы проекта с точки зрения пользователя, проверяем, что все работает так, как и задумывалось. 

### Зачем писать юнит-тесты

* Понять, что все работает
* Проверить что код ведет себя как надо и его поведение определено во всех случаях
* Подумать еще раз о крайних случаях, проверить их
* Проверить что ничего не сломалось после изменения кода/добавления новой функциональности
* Понять что именно сломалось, если сломалось 

## Doctest

Модуль doctest 
* находит части докстрингов, которые выглядят как интерактивные сессии питона 
* выполняет эти сессии
* проверяет, что результат выполнения кода соответствует написанному в документации

In [16]:
import datetime
import re
from typing import Optional


class RuDateParser:

    # Optional значит, что возвращаеся либо объект указанного типа, либо None
    def parse_numeric(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        if not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        try:
            result = datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None
        return result

In [17]:
parser = RuDateParser()
parser.parse_numeric('09-90-2010')

In [18]:
import doctest

In [19]:
doctest.testmod()

TestResults(failed=0, attempted=3)

## Юнит-тесты: unittest

* каждый самосоятельный блок кода (класс, метод, функция) тестируется отдельно и изолированно
* проверяем пары ввод-вывод

Давайте напишем юнит-тесты для метода parse_numeric с использованием всторенной библиотеки unittest:
* тесты в unittest организуются в группы внутри класса TestCase
* обычно - один Test Case для каждого класса/функции (но не всегда)
* отдельный тест на каждый вариант развития событий 
* тесты оформляются в виде методов класса, названия методов начинаются со слова test
* можно писать свои вспомогательные методы (которые не являются тестами), главное, чтобы их называния не начинались с test

In [20]:
import unittest

In [21]:
class RuDateParserTestCase(unittest.TestCase):
    def setUp(self):
        self.parser = RuDateParser()

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
    # тестируем поведение при вводе строки, не содержащей дату в нужном формате
    def test_parse_numeric_unmatching_string(self):
        self.assertEqual(None, self.parser.parse_numeric('12/01/2020'))
        
    # тестируем поведение при неправильном типе входных данных
    def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)
        
    def test_parse_numeric_wrong_day(self):
        self.assertEqual(None, self.parser.parse_numeric('90-01-2020'))
    
    def test_parse_numeric_wrong_month(self):
        self.assertEqual(None, self.parser.parse_numeric('09-90-2020'))
    
    def test_parse_numeric_wrong_year(self):
        self.assertEqual(None, self.parser.parse_numeric('09-90-0000'))

        # запустить все тесты
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter

.......
----------------------------------------------------------------------
Ran 7 tests in 0.004s

OK


### Assert

* Стандартная инструкция assert - проверить, истинно ли какое-то выражение в питоне. 
* Если нет, то поднимается AssertionError.

In [10]:
a = 2
assert a == 1, 'сообщение которое выведется вместе с AssertionError'

AssertionError: сообщение которое выведется вместе с AssertionError

В тестах можно использовать встроенный assert или методы класса unittest.TestCase. Если условие под assert не выполняется, то тест помечается непройденным. Полный список в [документации](https://docs.python.org/3/library/unittest.html#unittest.TestCase). 
* ***assertEqual***_(first, second, msg=None)_  
    Test that first and second are equal. If the values do not compare equal, the test will fail.

* ***assertNotEqual***_(first, second, msg=None)_  
    Test that first and second are not equal. If the values do compare equal, the test will fail.

* ***assertTrue***_(expr, msg=None)_      
  ***assertFalse***_(expr, msg=None)_      
    Test that expr is true (or false).

* ***assertIs***_(first, second, msg=None)_   
  ***assertIsNot***_(first, second, msg=None)_    
    Test that first and second evaluate (or don’t evaluate) to the same object.

* ***assertIn***_(first, second, msg=None)_    
  ***assertNotIn***_(first, second, msg=None)_     
    Test that first is (or is not) in second.

* ***assertRaises***_(exception, callable, *args, **kwds)_   
    Test that an exception is raised when callable is called with any positional or keyword arguments that are also passed to assertRaises(). The test passes if exception is raised, is an error if another exception is raised, or fails if no exception is raised. To catch any of a group of exceptions, a tuple containing the exception classes may be passed as exception.

### setUp и tearDown

Вспомогательные методы, которые используются, чтобы настроить среду тестирования. Нужны для того, чтобы убедиться, что каждый тест работает изолированно и на него не влияет, например, результат работы прошлого теста
 * ***setUp()*** - вызывается перед запуском каждого теста.
 * ***tearDown()*** - вызывается после запуска каждого теста. 

In [ ]:
class MyTest(unittest.TestCase):

    # создать временную папку перед запуском теста
    def setUp(self):
        self.test_dir = TemporaryDirectory()

    # удалить папку после выполнения теста
    def tearDown(self):
        self.test_dir.cleanup()

    # дальше идут сами тесты, в которых что-то делается с этой папкой 

**Задание:** 
1. Подумать, какой ввод сломает метод parse_numeric (он выдаст ошибку, но не поднятую нами TypeError), что мы не учли при написании метода
2. Исправить метод  
3. Дописать соответсвующий тест
4. Запустить тесты не в тетрадке

## Запуск тестов 

### командная строка

+ все тесты в проекте: ```python -m unittest discover ```
+ все тесты в модуле: ``` python -m unittest tests.test_something ```
+ все тесты в классе (тест-кейсе): 
``` python -m unittest tests.test_something.SomeTestCase ```
+ с измерением coverage: \
``` coverage run -m unittest ... ``` \
затем ``` coverage report -m ```


Флаг ***-m*** означает, что интерпретор найдет модуль/пакет с нужным именем и запустит его как  [***\_\_main\_\_***](https://docs.python.org/3/library/__main__.html#module-__main__), расширение (***.py***) писать не нужно ([документация](https://docs.python.org/3/using/cmdline.html#cmdoption-m)).

### PyCharm

+ правой кнопкой мыши на папку/файл/класс, выбрать run 'Unittest in ... (with Coverage)'
![](https://webdevblog.ru/wp-content/uploads/2019/04/py_run_test_folder.png)
+ или нажать на зеленую стрелку рядом с тем местом, которое хотим запустить

## Хорошие тесты

+ один юнит-тест проверяет только один фрагмент кода
+ все юнит-тесты работают независимо. все тесты можно запустить по отдельности и в любом порядке
+ должны проверяться все варианты развития событий и все побочные эффекты

In [22]:
# объект который хранит состояние
class MemoryPlus:
    
    def __init__(self):
        self._memory = []
    
    def plus_ten(self, number):
        self._memory.append(number)
        return number + 10
    
    @property
    def memory(self):
        return self._memory

In [23]:
# плохие юнит-тесты - не смотрим на побочные эффекты (добавилось ли значение в memory)
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

.......
----------------------------------------------------------------------
Ran 7 tests in 0.008s

OK


In [25]:
# плохие юнит-тесты - test_show_memory зависит от результатов test_plus_ten, 
# отдельно не будет работать, в другом порядке не будет работать
# важное замечание - unittest запускает тесты в алфавитном порядке, а не в написанном
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11
        
    def test_get_memory(self):
        assert self.mem_plus.memory == [1]
        
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

F.......
FAIL: test_get_memory (__main__.MemoryPlusTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-605bfc7d2879>", line 14, in test_get_memory
    assert self.mem_plus.memory == [1]
AssertionError

----------------------------------------------------------------------
Ran 8 tests in 0.005s

FAILED (failures=1)


In [ ]:
# плохие юнит-тесты (хотя не настолько плохие как предыдущие) - напихали все в один тест
# код не выполнится дальше первого проваленного assert 
# и не получится узнать сразу не работает только добаление в memory или все остальное тоже
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        
        assert self.mem_plus.memory == [1]
        assert result == 11

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

In [ ]:
# нормальные юнит-тесты
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11
        
    def test_memory(self):
        self.mem_plus.plus_ten(1)
        assert self.mem_plus.memory == [1]

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

### Coverage

+  Значение coverage (покрытия кода тестами) - % строчек кода, которые были запущены во время выполнения тестов
+ 100% покрытие не гарантирует того, что тесты хорошие и достаточные
+ Но хорошие и достаточные тесты должны (в идеале) обеспечивать 100% покрытие
+ [Документация](https://coverage.readthedocs.io/en/coverage-5.4/)

In [26]:
!pip install coverage

**Задание:**
1. Реализовать метод parse_natural (по аналогии с parse_numeric), сделать так, чтобы он не ломался при любом вводе. 
2. Запустить doctest и проверить, что все хорошо. 
2. Написать к нему тесты используя unittest. 
4. Добиться 100% покрытия кода тестами (или близкого к 100)

In [ ]:
import datetime
import re
from typing import Optional

class RuDateParser:
    
    def parse_natural(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd MMMM yyyy на русском языке

        >>> RuDateParser().parse_natural('1 января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)

        >>> RuDateParser().parse_natural('1 Января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)

        >>> RuDateParser().parse_natural('01 января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)
        
        >>> RuDateParser().parse_natural('не дата')
        """
        pass

## Динамическая генерация тестов

+ Нужно сделать много однотипных тестов (одна и та же функция/метод и т.д.)
+ Отличаются только входное значение и ожидаемый результат
+ Лучше не писать кучу почти одинакового кода, а генерировать тесты динамически. 

In [1]:
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

In [2]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_one_word(self):
        self.assertEqual(('', 'Петр', ''), parse_name('Петр'))
    def test_two_words(self):
        self.assertEqual(('Петров', 'Петр', ''), parse_name('Петров Петр'))
    def test_three_words(self):
        self.assertEqual(('Петров', 'Петр', 'Петрович'), 
                         parse_name('Петров Петр Петрович'))
    def test_more_words(self):
        self.assertEqual(('Петрова', 'Анна Мария', 'Васильевна'), 
                         parse_name('Петрова Анна Мария Васильевна'))
    def test_no_words(self):
        self.assertEqual(('', '', ''), parse_name(''))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


Cпособ 1 - просто в цикле:
+ все данные проверяются в одном тесте
+ метод прекращает работу после первой AssertionError
+ все что дальше не проверяется 

In [4]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_valid_name_parsing(self):
        for parsed_name, name in [
            (('', 'Петр', ''), ('Петр')),
            (('Петров', 'Петр', ''), ('Петров Петр')),
            (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
            (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
            (('', '', ''), (''))]:
            self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

F
FAIL: test_valid_name_parsing (__main__.ParseNameTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-5419b0f05315>", line 10, in test_valid_name_parsing
    self.assertEqual(parsed_name, parse_name(name))
AssertionError: Tuples differ: ('Петров', 'Петр', '') != ('', 'ПетровПетр', '')

First differing element 0:
'Петров'
''

- ('Петров', 'Петр', '')
?         ----

+ ('', 'ПетровПетр', '')
?  ++++


----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


Cпособ 2 (он лучше) - c помощью пакета ***parametErized*** (не путать с parameTRized без e):
+ на каждую пару генерируется отдельный независимый тест
+ точно понятно, что именно пошло не так
+ один упавший тест не влияет на все остальные

In [5]:
!pip install parameterized
from parameterized import parameterized

In [9]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    @parameterized.expand(
        [(('', 'Петр', ''), ('Петр')),
         (('Петров', 'Петр', ''), ('Петров Петр')),
         (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
         (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
         (('', '', ''), (''))])
    def test_valid_name_parsing(self, parsed_name, name):
        self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


**Задание**:
   + переписать тесты для parse_numeric с использованием parameterized

In [8]:
import datetime
import re
from typing import Optional

class RuDateParser:

    def parse_numeric(self, date): 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        elif not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        try:
            return datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None

In [10]:
import unittest
class RuDateParserTestCase(unittest.TestCase):
    
    def setUp(self):
        self.parser = RuDateParser()
        
    def tearDown(self):
        pass

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
#     # тестируем поведение при вводе строки, не содержащей дату в нужном формате
#     def test_parse_numeric_unmatching_string(self):
#         self.assertEqual(None, self.parser.parse_numeric())
        
#     def test_parse_numeric_day_too_large(self):
#         self.assertEqual(None, self.parser.parse_numeric())
        
#     def test_parse_numeric_month_too_large(self):
#         self.assertEqual(None, self.parser.parse_numeric())
        
#     def test_parse_numeric_day_zero(self):
#         self.assertEqual(None, self.parser.parse_numeric())
        
#     def test_parse_numeric_month_zero(self):
#         self.assertEqual(None, self.parser.parse_numeric())
        
    @parameterized.expand([
        ('12/01/2020'),
        ('52-01-2020'),
        ('21-21-2020'),
        ('00-01-2020'),
        ('01-00-2020')
    ])
    def test_parse_numeric_invalid_date(self, date_input_str):
        self.assertEqual(None, self.parser.parse_numeric(date_input_str))
        
    # тестируем поведение при неправильном типе входных данных
    def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)

# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # colab

............
----------------------------------------------------------------------
Ran 12 tests in 0.005s

OK


## Что еще полезного есть в unittest

### Пропуск тестов
+ если какие-то тесты нужны/должны работать только при определенных условиях

In [ ]:
# # это не рабочий код, он для примера и пояснения
class MyTestCase(unittest.TestCase):
    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(mylib.__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        if not external_resource_available():
            self.skipTest("external resource not available")
        # test code that depends on the external resource
        pass

In [11]:
import unittest
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

class ParseNameTestCase(unittest.TestCase):
    @parameterized.expand(
        [(('', 'Петр', ''), ('Петр')),
         (('Петров', 'Петр', ''), ('Петров Петр')),
         (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
         (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
         (('', '', ''), (''))])
    def test_valid_name_parsing(self, parsed_name, name):
        self.assertEqual(parsed_name, parse_name(name))
        
    @unittest.skip("this feature is not implemented yet")    
    def test_wrong_type(self):
        self.assertEqual(('', '', ''), parse_name(None))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....s.......
----------------------------------------------------------------------
Ran 13 tests in 0.009s

OK (skipped=1)


## Структура проекта c тестами

+ все тесты в отдельной папке, название test или tests
+ лучше превратить в пакет - добавить \_\_init\_\_.py (можно пустой)
+ названия файлов с тестами начинаются с test_

```
├── project root directory      
   ├── main project directory
   │   ├── ...
   │   ├── ...
   │    
   └── tests
       ├── __init__.py
       ├── test_*.py
       └── test_*.py 

```